# IIW Agent Playground

Welcome to the OpenMined Aries ACA-Py Juypter notebook playground. Using these notebooks you will be able to control your ACA-Py instance to interact with other agents being used at IIW. This should include mobile agents, other agent instances of this playground, trinsic agents and PICO agent applications. Basically any agent that supports the same aries-rfs as ACA-Py.

This is an experiment and might not work but hopefully it will be fun!

It is a playground because we have only created the skeleton to get you started. You can customise your "SSI Application" however you see fit by writing custom code right here within the notebooks. We have developed a python wrapper for the ACA-Py swagger api called the [aries-basic-controller](https://pypi.org/project/aries-basic-controller/) and a full tutorial series explaining how to use it to interact with your agent to perform basic agent flows [here](https://github.com/OpenMined/PyDentity/tree/master/tutorials/aries-basic-controller) if you get stuck.

Your agent is currently pointing at the Sovrin StagingNet and is using default storage for an indy wallet which is mapped using docker volumes so it should persist across multiple ./manage up's. Additionally the agents endpoint is exposed using ngrok, however this has a timelimit and will be a different url every time the docker is restarted. This *might* mess up the ability to communicate over connections, so the best advice is just to leave it running. Currently the agent is configured with all the --accept flags set that ACA-Py supports, feel free to change this. See the docker-compose.yml and the startup.sh script for more details.

We also included an attachment protocol so if you need to send files securely during IIW give it a try.




## Initialise your controller

In [14]:
%autoawait
import time
import qrcode
from aiohttp import FormData
import asyncio
import base64
from IPython.display import Image
import requests
import json

from termcolor import colored,cprint
from attachment_controller.attachment_controller import AttachmentController
    
# Anyone know how to get env variables into juypter notebooks through docker easily?
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://iiw-agent:8021"
# You may have changed this
API_KEY = "iiw_api_123456789"

# Note AttachmentController just extends the basic controller to include the new attachment endpoint
agent_controller = AttachmentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


## Configure your handlers

This is how you respond to messages that your ACA-Py instance receives. Probably a good place to start if you want to customise your application.

In [15]:
# Receive basic messages
def messages_handler(payload):
    
    global active_connection_id
    active_connection_id = payload["connection_id"]
    print("Handle message", payload, active_connection_id)
    print(connection)
    


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

# Receive attachments
def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    file_content = base64.b64decode(data)
    filepath = f"received_files/{filename}"
    f = open(filepath,'wb')
    f.write(file_content)
    f.close()

attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}

# Issue credential messages
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

# Receive connection messages
def connections_handler(payload):
    global STATE
    global active_connection_id
    active_connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == "response":

        # Ensures connections moved to active
        loop = asyncio.get_event_loop()
        time.sleep(2)
        print(colored("Trust ping send to ConnectionId {0} to activate connection".format(connection_id), "blue", attrs=["bold"]))
        loop.create_task(agent_controller.messaging.trust_ping(connection_id, 'hello!'))

    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("ConnectionId: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



# Note: You could have more than one listener & handler for the same topic
connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

# Receive proof messages
def proof_handler(payload):
    print("Handle present proof")
    print(payload)

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}



loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener, attach_listener, cred_listener, proof_listener, connection_listener], defaults=False)

## Do you intend to issue credentials? If so execute this block to write a DID to the Sovrin StagingNet.

In [3]:
# Check if you have a public DID
response = await agent_controller.wallet.get_public_did()
print(response)
result = response['result']

if  result == None:
    # generate new DID
    response = await agent_controller.wallet.create_did()

    did_object = response['result']
    print("New DID", did_object)

    # write new DID to Sovrin Stagingnet
    url = 'https://selfserve.sovrin.org/nym'

    payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

    # Adding empty header as parameters are being sent in payload
    headers = {}

    r = requests.post(url, data=json.dumps(payload), headers=headers)

    # Accept the TAA
    response = await agent_controller.ledger.get_taa()
    TAA = response['result']['taa_record']
    TAA['mechanism'] = "service_agreement"

    await agent_controller.ledger.accept_taa(TAA)

    response = await agent_controller.wallet.assign_public_did(did_object["did"])
    print("Your new public DID: ", response["result"]["did"])
else:
    print("Public DID already exists {}".format(result["did"]))

{'result': {'did': 'TXVcz4HPr8Tb4A3PrYpzkr', 'verkey': 'FTXp3W5Q6YTwAUizwM4jyxUaxJM12frN8UcB9xYUHb8H', 'public': True}}
Public DID already exists TXVcz4HPr8Tb4A3PrYpzkr


## What Credentials do you intend to issue if any?

You could define your own scheme and write them to the ledger or write a credential definition for a schema that already exists on the ledger. Check out [IndyScan](https://indyscan.io/txs/SOVRIN_STAGINGNET/domain?page=1&pageSize=50&filterTxNames=[%22SCHEMA%22]&sortFromRecent=true) for some ideas.

We have already written a schema IIW Encounter to the ledger. Feel free to create a credential definition and start issuing these credentials to your connections. You can view it [here](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/152586).

In [4]:
#pre-ledgered IIW Encounter schema ID
# Attributes: name
iiw_schema_id = '7Rm6HjjfY3VZGkHFPFtFNC:2:IIW Encounter:0.0.1'
response = await agent_controller.definitions.write_cred_def(iiw_schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

TXVcz4HPr8Tb4A3PrYpzkr:3:CL:152586:default


##  OPTIONAL - You can also define whatever schema you like and start issuing these

Go crazy, just make sure you save your schema_id and credential_definintion_id's for later use

In [ ]:
# Define you schema name - must be unique on the ledger
schema_name = "<YOUR CUSTOM SCHEMA NAME>"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["<CUSTOMISE THIS>"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)

# You might want to rename this variable
your_schema_id = response["schema_id"]
print(schema_id)

response = await agent_controller.definitions.write_cred_def(schema_id)

# And this one.
your_cred_def_id = response["credential_definition_id"]
print(cred_def_id)

## OPTIONAL - Or write a credential definition for an existing schema of your choice

It must be on the Sovrin StagingNet

In [13]:
# Write as many credential definitions as you like.
# Remember to save the cred_def_id's though. You will need these to issue credentials
some_indy_schema_id = input("Enter the StagingNet schema id you wish to issue")

response = await agent_controller.definitions.write_cred_def(some_indy_schema_id)
some_cred_def_id = response["credential_definition_id"]
print(some_cred_def_id)
    

Enter the StagingNet schema id you wish to issuePHXuUDwX9NFUHWRYTXkEJP:2:Useless Attributes:1.0
TXVcz4HPr8Tb4A3PrYpzkr:3:CL:152606:default


## Create an Invitation

You may want to set an alias so you know who your connection represents.

Additionally if you are sharing this url in a chat for others to connect you might want to make it multi_use.


In [9]:
# Config
alias = "OM IIW Playground Connection"
multi_use = "true"

# Create Invitation
invite = await agent_controller.connections.create_invitation(alias=alias, multi_use=multi_use)
# invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite['invitation_url'])
inviteURL = invite['invitation_url']
print(invite["invitation"])

Connection message {'invitation_mode': 'multi', 'connection_id': '02860b34-562b-4282-9749-846e7e14bbde', 'invitation_key': '146sgDidpSuZmXH3SnmPZdw3P7rTmJWVv8rKkvMQRxtL', 'routing_state': 'none', 'alias': 'OM IIW Playground Connection', 'accept': 'auto', 'created_at': '2020-10-20 13:53:23.361796Z', 'initiator': 'self', 'updated_at': '2020-10-20 13:53:23.361796Z', 'state': 'invitation'} 02860b34-562b-4282-9749-846e7e14bbde
Connection ID 02860b34-562b-4282-9749-846e7e14bbde
Invitation
https://9778c2823583.ngrok.io?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNTQyMTcxOTQtNzY3MS00OGJkLWE2ZDctOGFmNWVhOTk2NDYwIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwczovLzk3NzhjMjgyMzU4My5uZ3Jvay5pbyIsICJyZWNpcGllbnRLZXlzIjogWyIxNDZzZ0RpZHBTdVptWEgzU25tUFpkdzNQN3JUbUpXVnY4cktrdk1RUnh0TCJdLCAibGFiZWwiOiAiSUlXIn0=
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '54217194-7671-48bd-a6d7-8af5ea996460', 'serviceEn

## Accept Invitation
Copy in an invitation from someone else and create a connection

In [16]:


# Copy in the invitation json object here
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3e8cec32-2234-4292-9e5b-c2295661e7de', 'label': 'IIW', 'recipientKeys': ['6NGA53v968F9Eua9PnP1q9y7MxNQd9cbwFkwDTxxS5MG'], 'serviceEndpoint': 'https://cfdea7a6418e.ngrok.io'} 
connection = await agent_controller.connections.accept_connection(invitation)
connection_id = connection["connection_id"]

Connection message {'state': 'invitation', 'created_at': '2020-10-20 14:32:39.656916Z', 'routing_state': 'none', 'invitation_mode': 'once', 'connection_id': '478e21bb-0832-45ca-9e06-f998db9fde2a', 'initiator': 'external', 'invitation_key': '6NGA53v968F9Eua9PnP1q9y7MxNQd9cbwFkwDTxxS5MG', 'accept': 'manual', 'updated_at': '2020-10-20 14:32:39.656916Z', 'their_label': 'IIW'} 2e9659a6-931a-4366-8266-44a952d74b47
Connection message {'my_did': 'UvXL8HthKQvZmiqK7TeVJc', 'state': 'request', 'created_at': '2020-10-20 14:32:39.656916Z', 'routing_state': 'none', 'invitation_mode': 'once', 'connection_id': '478e21bb-0832-45ca-9e06-f998db9fde2a', 'initiator': 'external', 'invitation_key': '6NGA53v968F9Eua9PnP1q9y7MxNQd9cbwFkwDTxxS5MG', 'accept': 'manual', 'updated_at': '2020-10-20 14:32:39.669207Z', 'request_id': '8b3c4690-84e3-4a01-addd-52b117b12cf7', 'their_label': 'IIW'} 2e9659a6-931a-4366-8266-44a952d74b47
Connection message {'my_did': 'UvXL8HthKQvZmiqK7TeVJc', 'state': 'response', 'created_at'

In [5]:
connections = await agent_controller.connections.get_connections()
print(connections)

{'results': [{'my_did': 'XTE59Srdz445G4P4ZHtDPq', 'state': 'active', 'created_at': '2020-10-20 13:55:15.552646Z', 'routing_state': 'none', 'invitation_mode': 'once', 'connection_id': '4cdf41b7-14ea-4c92-b817-ef7358b9de16', 'their_did': 'PoPUeJK14PAWrB3NvUmMGQ', 'initiator': 'external', 'invitation_key': '146sgDidpSuZmXH3SnmPZdw3P7rTmJWVv8rKkvMQRxtL', 'accept': 'manual', 'updated_at': '2020-10-20 13:55:18.300565Z', 'request_id': '47235337-7b44-4d49-b995-146c1167bc0d', 'their_label': 'IIW'}, {'my_did': 'PoPUeJK14PAWrB3NvUmMGQ', 'state': 'active', 'created_at': '2020-10-20 13:55:16.096960Z', 'routing_state': 'none', 'invitation_mode': 'once', 'connection_id': '47a0978b-edeb-4d0d-ae89-d7f96d3f3e72', 'their_did': 'XTE59Srdz445G4P4ZHtDPq', 'initiator': 'multiuse', 'invitation_key': '146sgDidpSuZmXH3SnmPZdw3P7rTmJWVv8rKkvMQRxtL', 'accept': 'auto', 'updated_at': '2020-10-20 13:55:18.485833Z', 'their_label': 'IIW'}, {'my_did': 'RwP5cQkGgKQYDNGwpRdLNg', 'state': 'active', 'created_at': '2020-10-

## Optionally generate a QrCode from url

In [ ]:
# Link for connection invitation
input_data = inviteURL
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

Image(width=400, filename='./issuer_agent_invite_QRcode.png')

## Send a Message

You may want to create a better way to manage your contacts

In [17]:
basic_message = "hello from Will"
response = await agent_controller.messaging.send_message(active_connection_id, basic_message)
print("BASIC MESSAGE - Alice -> Bob")
print(response)

BASIC MESSAGE - Alice -> Bob
{}


## Send Attachment

Will only work if you are communicating with an agent that has this protocol installed. Basically anyone running this demo.

You must have the file you want to send accessible from this notebook. Upload it into the home page - https://localhost:8888. Or just send the IIW image that is already there.

In [7]:
file = "iiw.png"
content_type = "image/png"
data = FormData()
data.add_field('file',
               open(file, 'rb'),
               filename=file,
               content_type=content_type)
response = await agent_controller.protocol.send_attachment(active_connection_id, data)
print('Attachment- Alice-> Bob')
print(response)

Attachment- Alice-> Bob
{'thread_id': '48f179ac-1548-478a-9169-83d0025f4893'}


## Issue Credential

Customise this however you like. 

In [10]:
name=input("Please enter your name: ")
credential_attributes = [
    {"name": "name", "value": name},
]

record = await agent_controller.issuer.send_credential(active_connection_id, iiw_schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Please enter your name: Will


CancelledError: 

## Request a Presentation

You will need to know the schema_id you wish to verify attributes from


In [ ]:
schema_id = 'EuEtnVakYFyBtGFT1nHYtH:2:SSI PyDentity Tutorial:0.0.1'

print("Request proof of Fullname and Age range from Identity Holder")
#Set some variables

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "name", "restrictions": []},
]

# if revocation:
#     req_attrs.append(
#         {
#             "name": "skill",
#             "restrictions": [{"schema_id": schema_id}],
#             "non_revoked": {"to": int(time.time() - 1)},
#         },
#     )


#Set predicates for Zero Knowledge Proofs
# req_preds = [
#     # test zero-knowledge proofs
#     {
#         "name": "age",
#         "p_type": ">=",
#         "p_value": 21,
#         "restrictions": [{"schema_id": schema_id}],
#     }
# ]

indy_proof_request = {
    "name": "Proof of Name",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
#     "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
#     },
}

# if revocation:
#     indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

## 6. Verify Proof Presentation

This is checking the signatures on the credentials presented against the credential schema and definition id stored and resolvable on the ledger. It is a bit of a big complicated object, so we show the common pattern for breaking it down, checking it's verified and accessing the data that has been presented.

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

### Verifying the State

Once verified through the agent framework api, the state should be 'verified', otherwise the presentation is invalid.

### Accessing the revealed attributes

In [ ]:
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(val['raw'])

### Accessing the self-attested attributes

In [ ]:
for (name, val) in verify['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

In [13]:
await agent_controller.terminate()